can't load coreml model by running xcode cuz it doesn't work and not resolved.
Then convert to onnx and then to pytorch since there's no direct way to do it.
Colab runs on linux so can't run coreml, which runs on mac
Then do on mac. Got some env problem.

In [5]:
# Install required libraries
!pip install torch torchvision
!pip install tqdm

# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn.utils.prune as prune
import torch.nn.functional as F
from tqdm import tqdm

learning_rate = 0.001
batch_size = 64
epochs = 2
hidden_size = 1024
num_workers = 4  # Number of workers for data loading

# Define the transformations
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Grayscale(num_output_channels=3),  # MNIST to 3 channels (for RGB)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

# Load ResNet50 model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 10)  # Modify last layer for MNIST
model.to(device)  # Move model to GPU if available

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



In [6]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# for epoch in range(epochs):
#     model.train()
#     running_loss = 0.0
#     correct = 0
#     total = 0
#     for images, labels in train_loader:
#         optimizer.zero_grad()

#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#         _, predicted = torch.max(outputs, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")


# Training loop with tqdm progress bar
# for epoch in range(epochs):
#     model.train()
#     running_loss = 0.0
#     correct = 0
#     total = 0
#     for i, (images, labels) in tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}"):
#         optimizer.zero_grad()

#         # Forward pass
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#         # Calculate accuracy
#         _, predicted = torch.max(outputs, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

#     print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, (images, labels) in tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}"):
        images, labels = images.to(device), labels.to(device)  # Move data to GPU

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")



# Save the trained model
torch.save(model.state_dict(), 'resnet50_mnist.pth')
print("Model saved.")

#7:58am started

Epoch 1/2: 100%|██████████| 938/938 [10:10<00:00,  1.54it/s]

Epoch [1/2], Loss: 0.1550, Accuracy: 95.33%



Epoch 2/2: 100%|██████████| 938/938 [10:10<00:00,  1.54it/s]


Epoch [2/2], Loss: 0.0616, Accuracy: 98.14%
Model saved.


In [26]:

# Load the saved model
model = models.resnet50(pretrained=False)  # Initialize the model again
model.fc = nn.Linear(model.fc.in_features, 10)  # Modify the last fully connected layer again
model.load_state_dict(torch.load('resnet50_mnist.pth'))  # Load the saved weights
model.eval()  # Set the model to evaluation mode

<ipython-input-26-cacb3ca82c44>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('resnet50_mnist.pth'))  # Load the saved weights


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [27]:
import os

def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (MB):', size/1e6)
    os.remove('temp.p')
    return size

In [28]:
f = print_size_of_model(model,"full unpruned")


model:  full unpruned  	 Size (MB): 94.40919


In [29]:

# # Print model parameters and buffers before pruning
# print("Before pruning:")
# print(model.named_parameters())
# print(model.named_buffers())

# # Apply pruning using L1Unstructured
# params_to_prune = [
#     (model.layer1[0], 'weight'),
#     (model.layer1[1], 'weight'),
#     (model.fc, 'weight')  # Prune the fully connected layer
# ]

# Apply global unstructured pruning
# prune.global_unstructured(
#     params_to_prune,
#     pruning_method=prune.L1Unstructured,
#     amount=0.33,
# )

# # Print model parameters and buffers after pruning
# print("After pruning:")
# print(model.named_parameters())
# print(model.named_buffers())


The error is occurring because Bottleneck blocks inside the ResNet architecture don't have a direct attribute weight. Instead, they contain multiple sub-layers (e.g., conv1, conv2, conv3), and you need to specify the weight attribute of these sub-layers for pruning. To perform unstructured pruning on the layer1 block, you should select the weights of these individual convolutional layers within the Bottleneck blocks.

In [30]:
print("Before pruning:")
# for name, param in model.named_parameters():
#     print(name, param.shape)
# for name, buffer in model.named_buffers():
#     print(name, buffer.shape)

# Apply pruning using L1Unstructured
params_to_prune = [
    (model.layer1[0].conv1, 'weight'),  # Prune the first conv layer in the first Bottleneck block of layer1
    (model.layer1[0].conv2, 'weight'),  # Prune the second conv layer in the first Bottleneck block of layer1
    (model.layer1[0].conv3, 'weight'),  # Prune the third conv layer in the first Bottleneck block of layer1
    (model.layer1[1].conv1, 'weight'),  # Prune the first conv layer in the second Bottleneck block of layer1
    (model.layer1[1].conv2, 'weight'),  # Prune the second conv layer in the second Bottleneck block of layer1
    (model.layer1[1].conv3, 'weight'),  # Prune the third conv layer in the second Bottleneck block of layer1
    (model.fc, 'weight')                # Prune the fully connected layer
]

# Apply global unstructured pruning
prune.global_unstructured(
    params_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.33,
)


Before pruning:


In [31]:
print("After pruning:")
print(model.named_parameters())
print(model.named_buffers())

After pruning:
<generator object Module.named_parameters at 0x7c0d4bc48040>
<generator object Module.named_buffers at 0x7c0d4bc48040>


In [32]:

# Print model parameters and buffers after pruning
print("\nAfter pruning:")
# for name, param in model.named_parameters():
#     print("parameter:",name, param.shape)
# for name, buffer in model.named_buffers():
#     print("buffer:",name, buffer.shape)


After pruning:


In [33]:
for p in params_to_prune :
    # p takes the form (module, 'weight')
    prune.remove(*p)

"""
Observe that model size is doubled. This is because mask buffers are stored in addition to the original parameters.
So we might want to convert to sparse representations when storing on disk.

First, we remove the reparameterization, i.e. make the pruning "permanent":
"""

'\nObserve that model size is doubled. This is because mask buffers are stored in addition to the original parameters. \nSo we might want to convert to sparse representations when storing on disk.\n\nFirst, we remove the reparameterization, i.e. make the pruning "permanent":\n'

In [34]:
p = print_size_of_model(model, "newly pruned")
print("{0:.2f} times smaller".format(f/p))

model:  newly pruned  	 Size (MB): 94.40919
1.00 times smaller


In [40]:
import torch

def calculate_sparsity(param):
    """
    Calculate the sparsity of a given parameter tensor.

    Args:
        param (torch.Tensor): The parameter tensor for which sparsity is calculated.

    Returns:
        float: The sparsity percentage (between 0 and 100).
    """
    total_elements = param.numel()  # Total number of elements in the tensor
    zero_elements = torch.sum(param == 0).item()  # Count of zero elements
    sparsity = (zero_elements / total_elements) * 100  # Calculate sparsity as a percentage
    return sparsity


def calculate_sparsity_overall(model, params_to_prune):
    """
    Calculate the overall sparsity for specific parameters in the model.

    Args:
        model (torch.nn.Module): The model containing the parameters.
        params_to_prune (list): List of tuples of (layer, 'attribute_name') indicating layers to be pruned.

    Returns:
        float: The overall sparsity percentage (between 0 and 100) for the specified parameters.
    """
    total_elements = 0
    zero_elements = 0

    for layer, attr in params_to_prune:
        param = getattr(layer, attr)  # Get the parameter (e.g., weight)
        total_elements += param.numel()  # Add to total count
        zero_elements += torch.sum(param == 0).item()  # Count zero elements

    overall_sparsity = (zero_elements / total_elements) * 100  # Calculate overall sparsity as a percentage
    return overall_sparsity


def calculate_sparsity_model(model):
    """
    Calculate the overall sparsity of all parameters in the model.

    Args:
        model (torch.nn.Module): The model containing the parameters.

    Returns:
        float: The overall sparsity percentage (between 0 and 100) across the entire model.
    """
    total_elements = 0
    zero_elements = 0

    for param in model.parameters():
        total_elements += param.numel()  # Total elements across all parameters
        zero_elements += torch.sum(param == 0).item()  # Count of zero elements across all parameters

    model_sparsity = (zero_elements / total_elements) * 100  # Calculate model-wide sparsity as a percentage
    return model_sparsity
# Assuming model and params_to_prune are defined and pruning has been applied
print("Sparsity of specific layer (model.layer1[0].conv1.weight):", calculate_sparsity(model.layer1[0].conv1.weight))
print("Overall sparsity of pruned layers:", calculate_sparsity_overall(model, params_to_prune))
print("Overall model sparsity:", calculate_sparsity_model(model))


Sparsity of specific layer (model.layer1[0].conv1.weight): 11.3525390625
Overall sparsity of pruned layers: 32.99967447916667
Overall model sparsity: 0.206812820626812


In [35]:
# Now, we can easily convert the parameters to sparse representations:

sd = model.state_dict()
for item in sd:
    # if 'weight' in item: # shortcut, this assumes you pruned (and removed reparameterization for) all weight parameters
    #     print("sparsifying", item)
    sd[item] = model.state_dict()[item].to_sparse()
sd

OrderedDict([('conv1.weight',
              tensor(indices=tensor([[ 0,  0,  0,  ..., 63, 63, 63],
                                     [ 0,  0,  0,  ...,  2,  2,  2],
                                     [ 0,  0,  0,  ...,  6,  6,  6],
                                     [ 0,  1,  2,  ...,  4,  5,  6]]),
                     values=tensor([-0.0241,  0.0240,  0.0032,  ..., -0.0450,  0.0107,
                                    -0.0123]),
                     size=(64, 3, 7, 7), nnz=9408, layout=torch.sparse_coo)),
             ('bn1.weight',
              tensor(indices=tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,
                                      14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
                                      28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
                                      42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
                                      56, 57, 58, 59, 60, 61, 62, 63]]),
   

In [37]:
# now, you can save the sparse model
torch.save(sd, "sparse_model.pt")

# measure new size on disk:
print(f'{os.path.getsize("sparse_model.pt")/1e6} MB')

# notice the new size is not actually 1/10 of the size... consider why this might be the case

844.768966 MB


In [39]:
sparse_sd = torch.load("sparse_model.pt")

new_model = models.resnet50(pretrained=False)
new_model.fc = nn.Linear(new_model.fc.in_features, 10)

# Populate the new model's state dict, converting sparse tensors back to dense
new_model.load_state_dict({k: (v if v.layout == torch.strided else v.to_dense()) for k, v in sparse_sd.items()})

# Verify that the new model has been loaded correctly
new_model.eval()
print("Sparsified ResNet model loaded and ready for inference.")

<ipython-input-39-9017aba8faf4>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sparse_sd = torch.load("sparse_model.pt")


Sparsified ResNet model loaded and ready for inference.



To read and load a .mlmodel file on Google Colab, you need to use Apple's Core ML framework, which requires macOS. However, since Colab runs on Linux, you can't directly load and run Core ML models. Instead, here are two options:

1) do it on mac

2) Convert the .mlmodel to an ONNX model, which you can then load on Colab.

Tried to do (2) but didn't work out so ignore. I just trained from scratch

In [41]:
# !pip install coremltools onnx


In [42]:
# import coremltools as ct
# import onnx

# # Load the .mlmodel file
# model = ct.models.MLModel(path)

# # Convert the Core ML model to ONNX
# onnx_model = ct.converters.onnx.convert(model)

# # Save the converted ONNX model
# onnx.save(onnx_model, "model_original.onnx")
